In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# load the data

data_path = r"D:\Uber Demand Prediction\Session 5 - Training a baseline\final_data.csv"

df = pd.read_csv(data_path, parse_dates=["tpep_pickup_datetime"])

# parse_dates parameter, automatically converts the specified columns into datetime format instead of keeping them as plain text (strings).

In [3]:
df

,tpep_pickup_datetime,region,total_pickups,avg_pickups
0,2016-01-01 00:00:00,0,58,58.0
1,2016-01-01 00:15:00,0,120,97.0
2,2016-01-01 00:30:00,0,149,123.0
3,2016-01-01 00:45:00,0,160,140.0
4,2016-01-01 01:00:00,0,187,161.0
...,...,...,...,...
262075,2016-03-31 22:45:00,29,14,16.0
262076,2016-03-31 23:00:00,29,17,16.0
262077,2016-03-31 23:15:00,29,18,17.0
262078,2016-03-31 23:30:00,29,13,15.0


In [4]:
# shape of the data

df.shape

(262080, 4)

In [5]:
# datatypes

df.dtypes

tpep_pickup_datetime    datetime64[ns]
region                           int64
total_pickups                    int64
avg_pickups                    float64
dtype: object

In [6]:
# check for missing values

df.isna().sum()

tpep_pickup_datetime    0
region                  0
total_pickups           0
avg_pickups             0
dtype: int64

In [7]:
# extract the day of week information
df['day_of_week'] = df['tpep_pickup_datetime'].dt.day_of_week

df['month'] = df['tpep_pickup_datetime'].dt.month

In [8]:
# set the datetime column as index
df.set_index('tpep_pickup_datetime', inplace=True)

In [9]:
df

,region,total_pickups,avg_pickups,day_of_week,month
tpep_pickup_datetime,,,,,
2016-01-01 00:00:00,0,58,58.0,4,1
2016-01-01 00:15:00,0,120,97.0,4,1
2016-01-01 00:30:00,0,149,123.0,4,1
2016-01-01 00:45:00,0,160,140.0,4,1
2016-01-01 01:00:00,0,187,161.0,4,1
...,...,...,...,...,...
2016-03-31 22:45:00,29,14,16.0,3,3
2016-03-31 23:00:00,29,17,16.0,3,3
2016-03-31 23:15:00,29,18,17.0,3,3


In [10]:
# create the region grouper

region_grp = df.groupby('region')
region_grp

In [14]:
# shifting periods

periods = list(range(1,5))
periods

[1, 2, 3, 4]

In [16]:
# generate the lag features

lag_features = region_grp['total_pickups'].shift(periods)

lag_features

,total_pickups_1,total_pickups_2,total_pickups_3,total_pickups_4
tpep_pickup_datetime,,,,
2016-01-01 00:00:00,NaN,NaN,NaN,NaN
2016-01-01 00:15:00,58.0,NaN,NaN,NaN
2016-01-01 00:30:00,120.0,58.0,NaN,NaN
2016-01-01 00:45:00,149.0,120.0,58.0,NaN
2016-01-01 01:00:00,160.0,149.0,120.0,58.0
...,...,...,...,...
2016-03-31 22:45:00,22.0,14.0,15.0,13.0
2016-03-31 23:00:00,14.0,22.0,14.0,15.0
2016-03-31 23:15:00,17.0,14.0,22.0,14.0


**Why Use groupby Before Lagging?**
1. Avoiding Data Leakage Between Groups

    - If we apply .shift() directly on the entire total_pickups column without groupby('region'), the lagged values could come from different regions.
    
    - This would introduce incorrect relationships in the data because past values from one region should not influence future values of another region.
    
    
2. Maintaining Temporal Consistency Within Groups

    - When we groupby('region'), we ensure that each region's total_pickups are shifted only within that region.
    
    - This is crucial in time-series data because past values from one region should not affect another.

In [17]:
# merge them with the original df

data = pd.concat([lag_features, df], axis=1)

data

,total_pickups_1,total_pickups_2,total_pickups_3,total_pickups_4,region,total_pickups,avg_pickups,day_of_week,month
tpep_pickup_datetime,,,,,,,,,
2016-01-01 00:00:00,NaN,NaN,NaN,NaN,0,58,58.0,4,1
2016-01-01 00:15:00,58.0,NaN,NaN,NaN,0,120,97.0,4,1
2016-01-01 00:30:00,120.0,58.0,NaN,NaN,0,149,123.0,4,1
2016-01-01 00:45:00,149.0,120.0,58.0,NaN,0,160,140.0,4,1
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4,1
...,...,...,...,...,...,...,...,...,...
2016-03-31 22:45:00,22.0,14.0,15.0,13.0,29,14,16.0,3,3
2016-03-31 23:00:00,14.0,22.0,14.0,15.0,29,17,16.0,3,3
2016-03-31 23:15:00,17.0,14.0,22.0,14.0,29,18,17.0,3,3


In [18]:

print("The shape of the df before merger ", df.shape)
print("The shape of the df after merger ", data.shape)

The shape of the df before merger  (262080, 5)
The shape of the df after merger  (262080, 9)


In [21]:
# rows having missing values

data.isna().any(axis=1).sum()

# 30 groups * 4 window lag

120

In [33]:
# drop the missing values

data.dropna(inplace=True)

In [35]:
data.isna().any(axis=1).sum()

0

In [36]:
mapper = {name: f"lag_{ind+1}" for ind, name in enumerate(data.columns[0:4])}

mapper

{'lag_1': 'lag_1', 'lag_2': 'lag_2', 'lag_3': 'lag_3', 'lag_4': 'lag_4'}

In [37]:
# replace the column names

data = data.rename(columns=mapper)

In [38]:
data.head()

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week,month
tpep_pickup_datetime,,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4,1
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4,1
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4,1
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4,1
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4,1


In [39]:
# shape of the data

data.shape

(261960, 9)

In [40]:
# number of rows in each month

data['month'].value_counts()

month
3    89280
1    89160
2    83520
Name: count, dtype: int64

In [41]:
data

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week,month
tpep_pickup_datetime,,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4,1
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4,1
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4,1
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4,1
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4,1
...,...,...,...,...,...,...,...,...,...
2016-03-31 22:45:00,22.0,14.0,15.0,13.0,29,14,16.0,3,3
2016-03-31 23:00:00,14.0,22.0,14.0,15.0,29,17,16.0,3,3
2016-03-31 23:15:00,17.0,14.0,22.0,14.0,29,18,17.0,3,3


In [42]:
data.loc[data['month'].isin([1,2]), 'lag_1':'day_of_week']

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [43]:
# split the data

trainset = data.loc[data['month'].isin([1,2]), "lag_1":"day_of_week"]

testset = data.loc[data['month'].isin([3]), "lag_1":"day_of_week"]

In [44]:
trainset

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [46]:
# save the train and test data

train_data_save_path = r"D:\Uber Demand Prediction\Session 5 - Training a baseline\train.csv"

test_data_save_path = r"D:\Uber Demand Prediction\Session 5 - Training a baseline\test.csv"

trainset.to_csv(train_data_save_path, index=True)
testset.to_csv(test_data_save_path, index=True)

In [47]:
# make X_train and y_train

X_train = trainset.drop(columns=['total_pickups'])

y_train = trainset['total_pickups']


In [48]:
X_train.shape

(172680, 7)

In [49]:

X_train

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4
...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,14.0,0


In [50]:

y_train

tpep_pickup_datetime
2016-01-01 01:00:00    187
2016-01-01 01:15:00    194
2016-01-01 01:30:00    180
2016-01-01 01:45:00    197
2016-01-01 02:00:00    185
                      ... 
2016-02-29 22:45:00     12
2016-02-29 23:00:00     17
2016-02-29 23:15:00     15
2016-02-29 23:30:00     15
2016-02-29 23:45:00     12
Name: total_pickups, Length: 172680, dtype: int64

In [51]:
# make X_test and y_test

X_test = testset.drop(columns=["total_pickups"])

y_test = testset["total_pickups"]

In [53]:
X_test.shape

(89280, 7)

In [54]:

X_test

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1
...,...,...,...,...,...,...,...
2016-03-31 22:45:00,22.0,14.0,15.0,13.0,29,16.0,3
2016-03-31 23:00:00,14.0,22.0,14.0,15.0,29,16.0,3
2016-03-31 23:15:00,17.0,14.0,22.0,14.0,29,17.0,3


In [55]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error

In [56]:
from sklearn import set_config

set_config(transform_output='pandas')

In [57]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [58]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [59]:

# train the model

lr = LinearRegression()

# fit on the training data
lr.fit(X_train_encoded, y_train)

LinearRegression()

In [60]:

# make predictions on the train data

y_pred_train = lr.predict(X_train_encoded)

In [61]:

# make predictions on the test data

y_pred_test = lr.predict(X_test_encoded)

In [62]:

# evaluate the baseline model

train_mape = mean_absolute_percentage_error(y_train, y_pred_train)

test_mape = mean_absolute_percentage_error(y_test, y_pred_test)

In [63]:

test_mape

0.0793479028546311

In [64]:

print(f"MAPE on trainset is {(train_mape * 100):.2f}%")
print(f"MAPE on testset is {(test_mape * 100):.2f}%")

MAPE on trainset is 8.78%
MAPE on testset is 7.93%
